## Simulation Experiments of mbst and Projected mbst

In [1]:
import pandas as pd
import numpy as np
import time
import math

## mbst objects
from mbst import *
## training functions
from train import *
## functions generating experiment cases
from exp import *

/Users/hanqiu/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### generating experiment data

In [2]:
N_sample = 100000 ## sample size
N_feats = 10 ## number of feature
N_trees_real = 100 ## number of boosted trees in the random mbst

np.random.seed(32) ## fixed seed for replications

## different specifications of base effect and treatment effects
exp_case = 7
if exp_case == 1:
    ## 1 * base effect + 0.25 * treatment effect
    theta_dim = 2
    theta_factors = np.array([1,0.25])
    base_effect_flag = True
elif exp_case == 2:
    ## 1 * base effect + 1 * treatment effect
    theta_dim = 2
    theta_factors = np.array([1,1])
    base_effect_flag = True
elif exp_case == 3:
    ## 1 * treatment effect (1) + 0.5 * treatment effect (2)
    theta_dim = 2
    theta_factors = np.array([1,0.5])
    base_effect_flag = False
elif exp_case == 4:
    ## 1 * base effect + 1 * treatment effect (1) + 0.5 * treatment effect (2)
    theta_dim = 3
    theta_factors = np.array([1,1,0.5])
    base_effect_flag = True
elif exp_case == 5:
    ## 1 * base effect + 0.5 * treatment effect (1) + 0.25 * treatment effect (2)
    theta_dim = 3
    theta_factors = np.array([1,0.5,0.25])
    base_effect_flag = True
elif exp_case == 6:
    ## 1 * base effect + 0.5 * treatment effect (1) + 1 * treatment effect (2)
    theta_dim = 3
    theta_factors = np.array([1,0.5,1])
    base_effect_flag = True
elif exp_case == 7:
    ## 0.25 * treatment effect (1) + 1 * treatment effect (2) + 0.5 * treatment effect (3)
    theta_dim = 3
    theta_factors = np.array([0.25,1,0.5])
    base_effect_flag = False

In [3]:
## generate experiment data according to parameters above
feat_list = [str(i) for i in range(N_feats)]
feat_dict = dict([(feat,idx) for idx,feat in enumerate(feat_list)])
X_ext = np.random.normal(size=(N_sample,len(feat_list)))
df_ext = pd.DataFrame(X_ext,columns=feat_list)
X_treat = np.random.normal(size=(N_sample,theta_dim))
if base_effect_flag:
    X_treat[:,0] = 1
_,_,splits_dict = find_splits_dict(df_ext,feat_list)
mbst_real = generate_random_theta_mbst(N_trees_real,feat_list,splits_dict,theta_dim,theta_factors)
theta_real = mbst_real.predict(X_ext,feat_dict)
y_real = np.sum(X_treat * theta_real,axis=-1)

### applying different estimation methods

#### 0. setting common training params

In [4]:
N_trees_train = 300 ## number of boosted trees during training
N_train = int(N_sample * 0.7) ## training samples

params = {'lam_reg':1,
          'learning_rate':0.1,
          'max_depth':4,
          'min_split_loss':0,
          'subsample':0.7,
          'min_childs':10}

params_xgb = {'reg_lambda':params.get('lam_reg',1),
              'learning_rate':params.get('learning_rate',0.1),
              'max_depth':params.get('max_depth',4),
              'subsample':params.get('subsample',0.7),
              'objective':'reg:squarederror'}

def linear_obj(theta,labels):
    ## MSE loss and corresponding grad / hess
    X_treat,y = labels
    y_pred = np.sum(X_treat*theta,axis=-1)
    ll = np.square(y_pred - y)
    loss = np.sqrt(np.mean(ll))
    grad = (y_pred - y).reshape((-1,1)) * X_treat
    hess = np.expand_dims(X_treat,axis=1) * np.expand_dims(X_treat,axis=2)
    return loss,grad,hess

#### 1.vanilla xgb

In [5]:
X_all = np.concatenate([X_ext,X_treat],axis=-1)
xg_dtrain = xgb.DMatrix(X_all[:N_train],label=y_real[:N_train])
xg_dall = xgb.DMatrix(X_all,label=y_real)

tt = time.time()
xgb_bst = xgb.train(params_xgb,xg_dtrain,num_boost_round=N_trees_train,verbose_eval=False)
print('train time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
y_pred = xgb_bst.predict(xg_dall)
print('y predict time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
X_all_new = np.concatenate([X_ext,X_treat],axis=-1)

N_treat_sample = 100 ## for local linear regression
X_treat_samples = X_treat[np.random.choice(len(X_treat),N_treat_sample,replace=False)]
y_pred_new = []
for X_treat_sample in X_treat_samples:
    X_all_new[:,-theta_dim:] = X_treat_sample
    y_pred_new.append(xgb_bst.predict(xgb.DMatrix(X_all_new)))
y_pred_new = np.stack(y_pred_new,axis=0)
A = np.matmul(X_treat_samples.T,X_treat_samples)
b = np.matmul(X_treat_samples.T,y_pred_new)
theta_pred = np.linalg.solve(A,b).T
print('theta predict time: {:.2f}s.'.format(time.time() - tt))

theta_pred_xgb_base,y_pred_xgb_base = theta_pred,y_pred

train time: 7.24s.
y predict time: 0.15s.
theta predict time: 14.13s.


#### 2. multi-stage xgb

In [6]:
X_ext_train = X_ext[:N_train]
X_treat_train = X_treat[:N_train]
y_remain_train = y_real[:N_train].copy()
xg_dall = xgb.DMatrix(X_ext)

theta_pred = []
for iter_ in range(theta_dim):
    header = 'stage {} '.format(iter_)
    w = X_treat_train[:,iter_]
    valid_idxs = (np.abs(w) > 1e-3)
    xg_dtrain = xgb.DMatrix(X_ext_train[valid_idxs],label=y_remain_train[valid_idxs]/w[valid_idxs],
                            weight=w[valid_idxs]**2)
    
    tt = time.time()
    xgb_bst = xgb.train(params_xgb,xg_dtrain,
                        num_boost_round=int(N_trees_train // theta_dim) + 1,verbose_eval=False)
    print(header + 'train time: {:.2f}s.'.format(time.time() - tt))

    tt = time.time()
    theta_iter_pred = xgb_bst.predict(xg_dall)
    theta_pred.append(theta_iter_pred)
    y_remain_train -= w * theta_iter_pred[:N_train]
    print(header + 'predict time: {:.2f}s.'.format(time.time() - tt))
    
theta_pred = np.stack(theta_pred,axis=-1)
y_pred = np.sum(X_treat*theta_pred,axis=-1)

theta_pred_xgb_mstage,y_pred_xgb_mstage = theta_pred,y_pred

stage 0 train time: 2.20s.
stage 0 predict time: 0.06s.
stage 1 train time: 2.34s.
stage 1 predict time: 0.07s.
stage 2 train time: 2.28s.
stage 2 predict time: 0.06s.


#### 3. grf

In [7]:
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
from rpy2.robjects import r as rapi
numpy2ri.activate()
grf = importr('grf')

In [8]:
X_ext_train = X_ext[:N_train]
X_treat_train = X_treat[:N_train]
y_remain_train = np.expand_dims(y_real[:N_train].copy(),axis=-1)

theta_pred = []
for iter_ in range(theta_dim):
    header = 'stage {} '.format(iter_)
    w = X_treat_train[:,[iter_]]
    valid_idxs = (np.abs(w[:,0]) > 1e-3)
    
    tt = time.time()
    if np.std(w) < 1e-3:
        grf_bst = grf.regression_forest(X_ext_train[valid_idxs],y_remain_train[valid_idxs],
                               num_trees=int(N_trees_train // theta_dim) + 1)
    else:
        grf_bst = grf.causal_forest(X_ext_train[valid_idxs],y_remain_train[valid_idxs],
                               w[valid_idxs],num_trees=int(N_trees_train // theta_dim) + 1)
    print(header + 'train time: {:.2f}s.'.format(time.time() - tt))

    tt = time.time()
    theta_iter_pred = rapi.predict(grf_bst,X_ext)['predictions']
    theta_pred.append(theta_iter_pred)
    y_remain_train -= w * np.expand_dims(theta_iter_pred[:N_train],axis=-1)
    print(header + 'predict time: {:.2f}s.'.format(time.time() - tt))
    
theta_pred = np.stack(theta_pred,axis=-1)
y_pred = np.sum(X_treat*theta_pred,axis=-1)

theta_pred_grf,y_pred_grf = theta_pred,y_pred

stage 0 train time: 50.41s.
stage 0 predict time: 2.14s.
stage 1 train time: 42.81s.
stage 1 predict time: 2.23s.
stage 2 train time: 40.52s.
stage 2 predict time: 1.85s.


#### 4. mbst

In [9]:
tt = time.time()
mbst,_ = train_multi_bst(params,df_ext.iloc[:N_train].copy(),labels=(X_treat[:N_train],y_real[:N_train]),
                         feat_list=feat_list,num_boost_round=N_trees_train,
                         obj_func=linear_obj,verbose=False)
print('train time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
theta_pred = mbst.predict(X_ext,feat_dict)
y_pred = np.sum(X_treat*theta_pred,axis=-1)
print('predict time: {:.2f}s.'.format(time.time() - tt))

theta_pred_mbst,y_pred_mbst = theta_pred,y_pred

train time: 160.38s.
predict time: 3.02s.


#### 5. projected mbst

In [10]:
## first, define a variety of direction finding methods
def find_direc_func_null(grad,hess,grad_func,mbst,train_info):
    dim = grad.shape[-1]
    direc = np.ones((dim,)) / dim
    return direc

def find_direc_func_grad(grad,hess,grad_func,mbst,train_info):
    direc = find_max_var_direc(grad)
    return direc

def find_direc_func_approx(grad,hess,grad_func,mbst,train_info):
    H_sum = np.sum(hess,axis=0)
    dtheta_est = np.linalg.solve(H_sum,grad.T).T
    direc = find_max_var_direc(dtheta_est)
    return direc

def find_direc_func_nag(grad,hess,grad_func,mbst,train_info,moment_beta=0.9):
    moment = moment_beta * train_info.get('grad_prev',np.zeros(grad.shape))
    grad_new,hess_new = grad_func(train_info.get('curr_theta',np.zeros(grad.shape)) \
                                  + train_info.get('delta_theta_mean',np.zeros(grad.shape)) \
                                  + moment * mbst.learning_rate)
    grad_new = grad_new + moment
    direc = find_max_var_direc(grad_new)
    return direc

In [11]:
mbst_projs = []
theta_pred_projs = []
y_pred_projs = []
name_projs = []

find_direc_funcs = [
#                  ('n',find_direc_func_null),
                 ('g',find_direc_func_grad),
                ('hg',find_direc_func_approx),
                 ('nag',find_direc_func_nag),
                ]

## then, iterate all possible projected mbst options
for corrected_name,corrected in [('n',False),('pc',True)]:
#     for centered in [('nc',False),('c',True)]:
    for centered_name,centered in [('c',True)]:
        for direc_name,find_direc_func in (find_direc_funcs):
            name = 'project mbst ({},{},{})'.format(corrected_name,centered_name,direc_name)
            print(name)
            
            tt = time.time()
            mbst,_ = train_project_bst(params,df_ext.iloc[:N_train].copy(),
                                     labels=(X_treat[:N_train],y_real[:N_train]),
                                     feat_list=feat_list,num_boost_round=N_trees_train,
                                       find_direc_func=find_direc_func,
                                       corrected=corrected,centered=centered,
                                     obj_func=linear_obj,verbose=False)
            print('train time: {:.2f}s.'.format(time.time() - tt))

            tt = time.time()
            theta_pred = mbst.predict(X_ext,feat_dict)
            y_pred = np.sum(X_treat*theta_pred,axis=-1)
            print('predict time: {:.2f}s.'.format(time.time() - tt))
            
            name_projs.append(name)
            mbst_projs.append(mbst)
            theta_pred_projs.append(theta_pred)
            y_pred_projs.append(y_pred)

project mbst (n,c,g)
train time: 58.91s.
predict time: 3.00s.
project mbst (n,c,hg)
train time: 56.00s.
predict time: 2.84s.
project mbst (n,c,nag)
train time: 59.40s.
predict time: 2.93s.
project mbst (pc,c,g)
train time: 111.39s.
predict time: 2.77s.
project mbst (pc,c,hg)
train time: 113.16s.
predict time: 3.27s.
project mbst (pc,c,nag)
train time: 118.67s.
predict time: 2.55s.


### evaluations

In [12]:
def rounding(l,decimals=4):
    return list(np.round(l,decimals=decimals))

method_names = ['mean baseline','xgb','multi-stage xgb','grf','mbst'] + name_projs
y_preds = [np.mean(y_real),y_pred_xgb_base,y_pred_xgb_mstage,y_pred_grf,y_pred_mbst] + y_pred_projs
theta_preds = [np.mean(theta_real,axis=0),theta_pred_xgb_base,theta_pred_xgb_mstage,
               theta_pred_grf,theta_pred_mbst] \
                + theta_pred_projs

y_train_rmse = [np.sqrt(np.mean(np.square(y_pred - y_real)[:N_train])) for y_pred in y_preds]
y_test_rmse = [np.sqrt(np.mean(np.square(y_pred - y_real)[N_train:])) for y_pred in y_preds]
theta_train_rmse = [np.sqrt(np.mean(np.square(theta_pred - theta_real)[:N_train],axis=0)) 
                    for theta_pred in theta_preds]
theta_test_rmse = [np.sqrt(np.mean(np.square(theta_pred - theta_real)[N_train:],axis=0)) 
                   for theta_pred in theta_preds]
theta_train_corr = [np.zeros((theta_dim,))] + \
                    [np.array([np.corrcoef(theta_pred[:,col][:N_train], 
                                          theta_real[:,col][:N_train])[0,1] for col in range(theta_dim)]) 
                    for theta_pred in theta_preds[1:]]
theta_test_corr = [np.zeros((theta_dim,))] + \
                    [np.array([np.corrcoef(theta_pred[:,col][N_train:], 
                                          theta_real[:,col][N_train:])[0,1] for col in range(theta_dim)]) 
                    for theta_pred in theta_preds[1:]]

err_df = pd.DataFrame({'y rmse train':rounding(y_train_rmse),'y rmse test':rounding(y_test_rmse),
                   'theta rmse train':rounding(theta_train_rmse),'theta rmse test':rounding(theta_test_rmse),
                   'theta corr train':rounding(theta_train_corr),'theta corr test':rounding(theta_test_corr),})
err_df = err_df.T
err_df.columns = method_names
display(err_df)

,mean baseline,xgb,multi-stage xgb,grf,mbst,"project mbst (n,c,g)","project mbst (n,c,hg)","project mbst (n,c,nag)","project mbst (pc,c,g)","project mbst (pc,c,hg)","project mbst (pc,c,nag)"
y rmse train,0.1147,0.0594,0.0413,0.0515,0.032,0.0358,0.0358,0.0358,0.0347,0.0348,0.0348
y rmse test,0.116,0.0656,0.046,0.0606,0.0359,0.0398,0.0398,0.0398,0.0388,0.0388,0.0386
theta rmse train,"[0.0182, 0.1044, 0.0391]","[0.0165, 0.0547, 0.0278]","[0.0127, 0.0388, 0.0179]","[0.014, 0.0508, 0.0239]","[0.0098, 0.0289, 0.0165]","[0.0113, 0.0297, 0.0214]","[0.0113, 0.0298, 0.0215]","[0.0113, 0.0299, 0.0213]","[0.0109, 0.0297, 0.0199]","[0.0109, 0.0297, 0.02]","[0.0108, 0.0297, 0.02]"
theta rmse test,"[0.0181, 0.1042, 0.0392]","[0.0164, 0.055, 0.0279]","[0.0126, 0.0397, 0.0182]","[0.014, 0.0527, 0.0245]","[0.0098, 0.0299, 0.0167]","[0.0113, 0.031, 0.0215]","[0.0112, 0.031, 0.0217]","[0.0113, 0.0311, 0.0214]","[0.0109, 0.031, 0.02]","[0.011, 0.0309, 0.0201]","[0.0108, 0.0308, 0.0201]"
theta corr train,"[0.0, 0.0, 0.0]","[0.5277, 0.8752, 0.7576]","[0.7222, 0.9323, 0.8964]","[0.6684, 0.8819, 0.817]","[0.8495, 0.9618, 0.9095]","[0.7945, 0.9592, 0.8409]","[0.797, 0.9591, 0.838]","[0.7963, 0.9587, 0.8423]","[0.8144, 0.9592, 0.8646]","[0.8111, 0.9593, 0.8632]","[0.8147, 0.9594, 0.8636]"
theta corr test,"[0.0, 0.0, 0.0]","[0.522, 0.8715, 0.7568]","[0.7207, 0.9281, 0.8926]","[0.6666, 0.869, 0.8054]","[0.8468, 0.9588, 0.9071]","[0.7917, 0.9552, 0.8395]","[0.7941, 0.9553, 0.8364]","[0.7919, 0.955, 0.8408]","[0.8104, 0.9553, 0.8635]","[0.8069, 0.9556, 0.8618]","[0.8119, 0.9559, 0.8626]"
